In [38]:
import pandas as pd
#import pycaret
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# EDA Libraries
import sweetviz as sv
from pandas_profiling import ProfileReport
import pandas_profiling

#ML Libraries
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm

#preprocessing libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import joblib
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reads project's classes
from lib.preprocessor import Preprocessor
from lib.analyser import Analyser
from lib.utils import Performance
from lib.trainer import Trainer

In [17]:
data = pd.read_csv("data/cup98VAL.csv",na_values=" ",index_col='CONTROLN')
target = pd.read_csv("data/valtargt.csv",index_col='CONTROLN')
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,CLUSTER,AGE,AGEFLAG,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
CONTROLN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
188946,9301,PV3,0,WI,53558,NaN,NaN,1702,0.0,X,NaN,NaN,NaN,XXXX,T1,35.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,F,9.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0,19,40,28,6,11,5,NaN,NaN,NaN,8.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2169,599,747,99,0,0,50,50,99,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,32,39,40,31,42,44,31,39,43,18,10,24,31,16,8,7,2,14,17,38,19,12,13,4,12,15,31,54,36,13,2,70,9,2,19,223,290,88,82,6,11,4,3,0,0,0,87

In [18]:
data.shape

(96367, 478)

In [19]:
# Gets some redundant variables based on variance, sparsity & common sense
redundant_vars = Analyser.get_redundant_vars(data)

# Drops redundant cols/
data = data.drop(redundant_vars, axis = 1)

# Gets columns which have high nan values and drop them
nan_cols = Analyser.get_nan_cols(data)

# Drops redundant cols/
data = data.drop(nan_cols, axis = 1)

# Gets categorical columns which have high cardinality values and drop them
cardinality_vars = Analyser.get_high_cardinality_vars(data)

# Drops redundant cols/
data = data.drop(cardinality_vars, axis = 1)

# Imputes the data and fills in the missing values
data = Preprocessor.fill_nans(data)

#Get dummy files 
df = pd.get_dummies(data)



#Lets normalize the data before PCA as a prerequisite
scaler = StandardScaler()
df = scaler.fit_transform(df)


#Lets create PCA from the scaled data
pca = PCA(n_components=50)
principalComponents = pca.fit_transform(df)
df = pd.DataFrame(data = principalComponents,index = data.index)

In [20]:
print(data.shape,df.shape)

(96367, 373) (96367, 50)


In [40]:

# load the model from disk to find donors
filename='models/donor_model.sav'
donor_model = joblib.load(filename)


# load the model from disk to find donation amout
filename='models/donations_model.sav'
donation_model = joblib.load(filename)

# load the rfe_features from disk
filename='models/donations_model_features.csv'
col = joblib.load(filename)

In [22]:
#predict the donors
df['prediction'] = donor_model.predict(df)  # to predict donations of only positive donors


In [23]:
donors = df[df['prediction']==1]

In [24]:
#only using the rfe features for prediction
donors = donors[col]
df['donation'] = donation_model.predict(sm.add_constant(donors))

In [25]:
predictions= df[['prediction','donation']]

#filling predictions as 0 for non-donors
predictions = predictions.fillna(0)

In [27]:
result = target.join(predictions)
result

In [29]:
Performance.get_perf(result['TARGET_B'], result['prediction'])

[0.6294063320431268,
 0.4890211368766674,
 0.06692692242880413,
 0.11774006274858569]

Performance metrics on validation data
- Accuracy of 62%
- Recall of ~49%
- Precision of ~7%

In [101]:
#Missed Donation due to wrong prediction
result[(result['TARGET_D']>0) &(result['prediction']==0)].TARGET_D.sum()

50864.95

In [102]:
#Donations received and predicted correctly
result[(result['TARGET_B']>0) &(result['prediction']==1)].TARGET_D.sum()

25224.69

In [35]:
#Number of Wrong donors selected
result[(result['TARGET_B']==0) &(result['prediction']==1)].shape

(33223, 4)

In [36]:
#Number of right donors selected
result[(result['TARGET_B']==1) &(result['prediction']==1)].shape

(2383, 4)

In [37]:
#Number of right donors missed
result[(result['TARGET_B']==1) &(result['prediction']==0)].shape

(2490, 4)

In [32]:
result.shape

(96367, 4)

In [99]:
result[(result['TARGET_D']>0)]

,TARGET_B,TARGET_D,prediction,donation
CONTROLN,,,,
25,1,25.0,1,20.488776
50,1,10.0,1,10.913321
57,1,8.0,1,14.096741
74,1,15.0,0,0
90,1,10.0,1,2.190867
...,...,...,...,...
191521,1,10.0,0,0
191522,1,10.0,0,0
191555,1,15.0,0,0


## OUTCOME

- The Model does not perform very well on the given data and missed on right donors and quite an amount of Donations as well

- The reason may be poor quality of data or insufficient amount of time spent in Features engineering and understanding the features as all the models are performing in similar way. 